# Exercise
file name: clustering_anomaly_detection.py or clustering_anomaly_detection.ipynb
# Clustering - DBSCAN
# Ideas:
- Use DBSCAN to detect anomalies in curriculumn access.
- Use DBSCAN to detect anomalies in other products from the customers dataset.
- Use DBSCAN to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror).

# 1. Use DBSCAN to detect anomalies in curriculumn access.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# DBSCAN import
from sklearn.cluster import DBSCAN

# Scaler import
from sklearn.preprocessing import MinMaxScaler

import env

In [2]:
def get_curriculum_logs():
    filename = "curriculum-access.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col=False)
    else:
        # read the SQL query into a dataframe
        url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
        query = '''
        SELECT date,
               path as endpoint,
               user_id,
               cohort_id,
               ip as source_ip
        FROM logs;
        '''
        df = pd.read_sql(query, url)

        # Write that dataframe to disk for later.
        df.to_csv(filename, index = False)

        return df  

In [3]:
def anomalies_curriculum_access():
    # acquire data using the above function
    df = get_curriculum_logs()

    # convert date to a pandas datetime format and set as index
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)

    page_views = df.groupby(['user_id'])['endpoint'].agg(['count', 'nunique'])

    # create the scaler
    scaler = MinMaxScaler().fit(page_views)
    # use the scaler
    page_views_scaled_array = scaler.transform(page_views)

    # construct DBSCAN object
    dbsc = DBSCAN(eps = 0.1, min_samples=4).fit(page_views_scaled_array)

    # Now, let's add the scaled value columns back onto the dataframe
    columns = list(page_views.columns)
    scaled_columns = ["scaled_" + column for column in columns]

    # Create a dataframe containing the scaled values
    scaled_df = pd.DataFrame(page_views_scaled_array, columns=scaled_columns, index=page_views.index)

    # Merge the scaled and non-scaled values into one dataframe
    page_views = page_views.merge(scaled_df, left_index=True, right_index=True)
    
    # defining labels
    labels = dbsc.labels_

    #add labels back to the dataframe
    page_views['labels'] = labels

    # anomalies 

    anomalies_df = page_views[page_views.labels==-1]

    return anomalies_df

In [4]:
anomalies_curriculum_access()

,count,nunique,scaled_count,scaled_nunique,labels
user_id,,,,,
1,7404,976,0.413298,1.000000,-1
11,17913,871,1.000000,0.892308,-1
53,12329,333,0.688254,0.340513,-1
64,16347,437,0.912573,0.447179,-1
248,5075,625,0.283274,0.640000,-1


# 2. Use DBSCAN to detect anomalies in other products from the customers dataset.

In [5]:
from env import host, user, password

def get_db_url(database, host=host, user=user, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'


In [6]:
url = get_db_url("grocery_db")

sql = """
select *
from grocery_customers
"""

df = pd.read_sql(sql, url, index_col="customer_id")
df.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
customer_id,,,,,,,,
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


In [7]:
df =  df[['Grocery', 'Frozen']]
df.head()

,Grocery,Frozen
customer_id,,
0,7561,214
1,9568,1762
2,7684,2405
3,4221,6404
4,7198,3915


In [8]:
def anomalies_customers():
    # acquire data using the above function
    df = pd.read_sql(sql, url, index_col="customer_id")

    # selecting frozen and Grocery
    df =  df[['Grocery', 'Frozen']]

    # create the scaler
    scaler = MinMaxScaler().fit(df)
    # use the scaler
    df_scaled_array = scaler.transform(df)

    # construct DBSCAN object
    dbsc = DBSCAN(eps = 0.1, min_samples=4).fit(df_scaled_array)

    # Now, let's add the scaled value columns back onto the dataframe
    columns = list(df.columns)
    scaled_columns = ["scaled_" + column for column in columns]

    # Create a dataframe containing the scaled values
    scaled_df = pd.DataFrame(df_scaled_array, columns=scaled_columns, index=df.index)

    # Merge the scaled and non-scaled values into one dataframe
    df = df.merge(scaled_df, left_index=True, right_index=True)
    
    # defining labels
    labels = dbsc.labels_

    #add labels back to the dataframe
    df['labels'] = labels

    # anomalies 

    anomalies_df = df[df.labels==-1]

    return anomalies_df

In [9]:
anomalies_customers()

,Grocery,Frozen,scaled_Grocery,scaled_Frozen,labels
customer_id,,,,,
47,55571,7782,0.598942,0.127490,-1
61,59598,3254,0.642347,0.053070,-1
65,45828,36,0.493926,0.000181,-1
85,92780,1026,1.000000,0.016452,-1
93,2062,35009,0.022193,0.574979,-1
183,20170,36534,0.217371,0.600043,-1
325,13626,60869,0.146836,1.000000,-1
333,67298,131,0.725341,0.001742,-1


In [10]:
def anomalies_customers(col1, col2, ep_value, min_sample_value):
    # acquire data using the above function
    df = pd.read_sql(sql, url, index_col="customer_id")

    # selecting col2 and col1
    df =  df[[col1, col2]]

    # create the scaler
    scaler = MinMaxScaler().fit(df)
    # use the scaler
    df_scaled_array = scaler.transform(df)

    # construct DBSCAN object
    dbsc = DBSCAN(eps = ep_value, min_samples=min_sample_value).fit(df_scaled_array)

    # Now, let's add the scaled value columns back onto the dataframe
    columns = list(df.columns)
    scaled_columns = ["scaled_" + column for column in columns]

    # Create a dataframe containing the scaled values
    scaled_df = pd.DataFrame(df_scaled_array, columns=scaled_columns, index=df.index)

    # Merge the scaled and non-scaled values into one dataframe
    df = df.merge(scaled_df, left_index=True, right_index=True)
    
    # defining labels
    labels = dbsc.labels_

    #add labels back to the dataframe
    df['labels'] = labels

    # anomalies 

    anomalies_df = df[df.labels==-1]

    return anomalies_df

In [11]:
anomalies_customers('Grocery', 'Frozen', 0.1, 4)

,Grocery,Frozen,scaled_Grocery,scaled_Frozen,labels
customer_id,,,,,
47,55571,7782,0.598942,0.127490,-1
61,59598,3254,0.642347,0.053070,-1
65,45828,36,0.493926,0.000181,-1
85,92780,1026,1.000000,0.016452,-1
93,2062,35009,0.022193,0.574979,-1
183,20170,36534,0.217371,0.600043,-1
325,13626,60869,0.146836,1.000000,-1
333,67298,131,0.725341,0.001742,-1


# 3. Use DBSCAN to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror).

In [15]:
import acquire
import prepare

In [16]:
# Below code is what is used in the prepare module to acquire the data before preparing
df= acquire.new_zillow_data()

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   52441 non-null  float64
 1   bedroomcnt                    52441 non-null  float64
 2   calculatedfinishedsquarefeet  52359 non-null  float64
 3   fips                          52441 non-null  float64
 4   lotsizesquarefeet             52072 non-null  float64
 5   poolcnt                       11096 non-null  float64
 6   regionidzip                   52415 non-null  float64
 7   yearbuilt                     52325 non-null  float64
 8   latitude                      52441 non-null  float64
 9   longitude                     52441 non-null  float64
 10  taxvaluedollarcnt             52440 non-null  float64
 11  garagetotalsqft               18015 non-null  float64
 12  regionidcounty                52441 non-null  float64
 13  r

In [18]:
# Acquire and Prepare zillow data
df = prepare.wrangle_zillow(df)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51883 entries, 0 to 52440
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   51883 non-null  float64
 1   bedroomcnt                    51883 non-null  float64
 2   calculatedfinishedsquarefeet  51883 non-null  float64
 3   fips                          51883 non-null  float64
 4   lotsizesquarefeet             51883 non-null  float64
 5   poolcnt                       51883 non-null  float64
 6   regionidzip                   51883 non-null  float64
 7   yearbuilt                     51883 non-null  float64
 8   latitude                      51883 non-null  float64
 9   longitude                     51883 non-null  float64
 10  taxvaluedollarcnt             51883 non-null  float64
 11  garagetotalsqft               51883 non-null  float64
 12  regionidcounty                51883 non-null  float64
 13  r

In [20]:
def anomalies_zillow(col1, col2, ep_value, min_sample_value):
    # acquire data using the above function
    df= acquire.new_zillow_data()
    
    # preparing df
    df = prepare.wrangle_zillow(df)

    # selecting col2 and col1
    df =  df[[col1, col2]]

    # create the scaler
    scaler = MinMaxScaler().fit(df)
    # use the scaler
    df_scaled_array = scaler.transform(df)

    # construct DBSCAN object
    dbsc = DBSCAN(eps = ep_value, min_samples=min_sample_value).fit(df_scaled_array)

    # Now, let's add the scaled value columns back onto the dataframe
    columns = list(df.columns)
    scaled_columns = ["scaled_" + column for column in columns]

    # Create a dataframe containing the scaled values
    scaled_df = pd.DataFrame(df_scaled_array, columns=scaled_columns, index=df.index)

    # Merge the scaled and non-scaled values into one dataframe
    df = df.merge(scaled_df, left_index=True, right_index=True)
    
    # defining labels
    labels = dbsc.labels_

    #add labels back to the dataframe
    df['labels'] = labels

    # anomalies 

    anomalies_df = df[df.labels==-1]

    return anomalies_df

In [21]:
anomalies_zillow('bedroomcnt', 'calculatedfinishedsquarefeet', 0.1, 4)

,bedroomcnt,calculatedfinishedsquarefeet,scaled_bedroomcnt,scaled_calculatedfinishedsquarefeet,labels
4864,8.0,21929.0,0.7,1.000000,-1
16571,7.0,20612.0,0.6,0.939523,-1
21981,11.0,4012.0,1.0,0.177251,-1
27017,6.0,13598.0,0.5,0.617440,-1
28105,6.0,15450.0,0.5,0.702484,-1
33311,10.0,4294.0,0.9,0.190201,-1
48495,10.0,17245.0,0.9,0.784911,-1
